In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
df_train = pd.read_csv("train2.txt",sep=",")
df_test = pd.read_csv("test2.txt",sep=",")

In [ ]:
df_train

In [ ]:
df_test

In [ ]:
# df_train.info()
df_train['age'].unique() #there are 2 nan value; 50-59, 60-69,40-49,20-29,70-79,30-39,60-60,80-89,0-10,0-6
# df_train['sex'].unique()# there 5 different sex 
# df_train['city'].unique() # doesn't exist nan
# df_train['province'].unique() # doesn't exist nan
# df_train['country'].unique() # doesn't exist nan
# df_train['symptoms'].unique()
# df_train['duration'].unique() # int64


In [ ]:
# df_test.info()
df_test['age'].unique() #there are 4 nan value; 30-39，50-59，60-69，40-49，40-49，80-89，0-10
# df_test['sex'].unique()# there 5 different sex 
# df_test['city'].unique() # doesn't exist nan
# df_test['province'].unique() # doesn't exist nan
# df_test['country'].unique() # doesn't exist nan
# df_test['symptoms'].unique()
# df_test['duration'].unique() # int64


In [ ]:
#check_train_nan=df_train['age'].isnull().sum()
#print(check_train_nan)
#check_test_nan=df_test['age'].isnull().sum()
#print(check_test_nan)

df_train['age'].unique()

In [ ]:
df_test['age'].unique()

In [ ]:
## replace all age range to integer number
df_train['age']=df_train['age'].replace(['60-60'],'60')
df_train['age'].unique()

In [ ]:
import pandas as pd
from collections import defaultdict
import numpy as np
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import make_regression
from sklearn.neural_network import MLPRegressor
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_friedman2
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF
from sklearn.linear_model import BayesianRidge, LinearRegression


In [ ]:
def cal_time(x):
    return (x - pd.Timestamp('2020-01-01'))

def cal_average(agerange):
    
    if type(agerange) is str and '-' in agerange: 
        age = agerange.split('-')
        return (int(age[0])+int(age[1]))/2
    else:
        if type(agerange) == float and math.isnan(agerange):
            return 50
        return agerange

def label_helper(series):
    dic = {}
    counter = 0
    for s in series:
        if s in dic.keys():
            continue
        else:
            dic[s] = str(counter)
            counter += 1
    return dic

def convert_label(df, headers):
    for i in range(len(headers)):
        h = headers[i]
        df[h] = df[h].apply(label_helper(df[h]).get)


In [ ]:
train = pd.read_csv('train_clean_dataset.csv')
test = pd.read_csv('test_clean_dataset.csv')

train['confirmed'] = pd.to_datetime(train['confirmed'], format='%d.%m.%Y')
train['confirmed'] = train['confirmed'].apply(cal_time).dt.days

test['confirmed'] = pd.to_datetime(test['confirmed'], format='%d.%m.%Y')
test['confirmed'] = test['confirmed'].apply(cal_time).dt.days


In [ ]:
label_transform('sex')
label_transform('city')
label_transform('province')
label_transform('country')
label_transform('V1')
train['confirmed'] = train['confirmed'].apply(cal_average)
test['confirmed'] = test['confirmed'].apply(cal_average)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train.loc[:, (train.columns != 'duration') 
                                                              & (train.columns != 'Id') 
                                                            ], train['duration'], 
                                                    test_size=0.005, random_state=200)
test_train = test.loc[:,  (test.columns != 'Id')]



In [ ]:
kernel = DotProduct() +1 * RBF(1.0)
gpr = GaussianProcessRegressor(kernel=kernel,random_state=0).fit(X_train, y_train)
print('gpr score: '+ str(gpr.score(X_train, y_train)))
predict_result_gussian = gpr.predict(X_test, return_std=True)


regr = svm.SVR(kernel='linear').fit(X_train, y_train)
predict_result = regr.predict(X_test)
mse = (mean_squared_error(list(predict_result_gussian[0]), y_test))**(1/2)

In [ ]:
predict_result_gussian = gpr.predict(test_train, return_std=True)
result_df = pd.DataFrame(columns=['Id', 'duration'])
result_df['Id'] = test['Id']+1
result_df['duration'] = predict_result_gussian[0]
result_df.to_csv('result_guassian.csv',index=False)